<a href="https://colab.research.google.com/github/Logan-Stark/DS-Unit-2-Kaggle-Challenge/blob/master/module3-cross-validation/LS_DS_223_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


**You can't just copy** from the lesson notebook to this assignment.

- Because the lesson was **regression**, but the assignment is **classification.**
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [2]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [3]:
import category_encoders as ce
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from category_encoders import OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
# Split train into train & val
#train, test = train_test_split(train, train_size=0.80, test_size=0.20, 
                             # stratify=train['status_group'], random_state=42)

In [5]:
# added new features to our data and got rid of duplicates
import numpy as np

def wrangle(X):
    """Wrangle train, validate, and test sets in the same way"""
    
 
    # Stop CopyWarning
    X = X.copy()
    
    # get rid of irrelevant Latitude data
    X['latitude'] = X['latitude'].replace(-2e-08, 0)
    
    # we will replace the zeros with nulls, and impute missing values later.
    cols_with_zeros = ['longitude', 'latitude', 'num_private', 'district_code'
    ,'population', 'construction_year', 'gps_height', 'amount_tsh']
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)

    # converting date recorderd into int and making new feature
    X['date_recorded'] = pd.to_datetime(X['date_recorded'])
    X['date_recorded'] = X['date_recorded'].dt.year
    X['date_recorded'] = X['date_recorded'].astype(int)
    X['years_since_constr'] = X['date_recorded'] - X['construction_year']   
    
    # making a feature based on correlation between salty and other for water quality and extraction
    X['water_quality_extraction_method']= (X['water_quality']== 'salty') & (X['extraction_type_class'] == 'other')

    # making feature based on correlation between salty and dry for water quality and quantity dry
    X['quality_and_quantity'] = (X['water_quality'] == 'salty') | (X['quantity']== 'dry')

    # making feature based on correlation between salty and dry for water quality and quantity dry
    X['Year_built_insufficientsupply'] = (X['quantity']=='insufficient') | (X['construction_year']<= 2004)

    # making feature based on correlation between salty and dry for water quality and quantity dry
    X['Year_built_sufficientsupply'] = (X['quantity']=='enough') | (X['construction_year']>= 2004)

    # making feature based on correlation between gravity and insufficent
    X['extraction_gravity_insufficent'] = (X['quantity']=='insufficient') & (X['extraction_type_class'] == 'gravity')

    #X['gps_enough']= (X['gps_height']>= 1550) & (X['quantity_group']== 'enough')

    # quantity & quantity_group are duplicates, so drop one
    X = X.drop(columns='quantity')
    X = X.drop(columns='extraction_type_group')
    X = X.drop(columns='management_group')
    X = X.drop(columns='payment_type')
    X = X.drop(columns='quality_group')
    X = X.drop(columns='source_type')
    X = X.drop(columns='waterpoint_type_group')
    X = X.drop(columns='extraction_type')
    X = X.drop(columns='num_private')
    X = X.drop(columns='scheme_name')
    # return the wrangled dataframe
    return X

# DRY programming DON'T REPEAT YOURSELF
train = wrangle(train)
test = wrangle(test)

In [6]:
# select our target 
target = 'status_group'

# make train without our target or id
train_features = train.drop(columns=[target, 'id'])

#make numeric features
numeric_features = train_features.select_dtypes(include='number').columns.tolist()

# make a cardinality feature to help filter
cardinality = train_features.select_dtypes(exclude='number').nunique()

# get a list of relevant categorical data
categorical_features = cardinality[cardinality <=50].index.tolist()

# Combine the lists 
features = numeric_features + categorical_features
print(features)

['amount_tsh', 'date_recorded', 'gps_height', 'longitude', 'latitude', 'region_code', 'district_code', 'population', 'construction_year', 'years_since_constr', 'basin', 'region', 'public_meeting', 'recorded_by', 'scheme_management', 'permit', 'extraction_type_class', 'management', 'payment', 'water_quality', 'quantity_group', 'source', 'source_class', 'waterpoint_type', 'water_quality_extraction_method', 'quality_and_quantity', 'Year_built_insufficientsupply', 'Year_built_sufficientsupply', 'extraction_gravity_insufficent']


In [7]:
nonfunc = (train[train['status_group']== 'non functional']) 

In [8]:
nonfunc['district_code'].value_counts()

1.0     4358
3.0     4345
2.0     4126
4.0     2895
6.0     1787
5.0     1683
7.0     1115
33.0     582
53.0     469
8.0      401
43.0     284
13.0     243
30.0     220
63.0     128
23.0     102
62.0      52
60.0       9
80.0       8
Name: district_code, dtype: int64

In [9]:
train['district_code'].value_counts()

1.0     12203
2.0     11173
3.0      9998
4.0      8999
5.0      4356
6.0      4074
7.0      3343
8.0      1043
30.0      995
33.0      874
53.0      745
43.0      505
13.0      391
23.0      293
63.0      195
62.0      109
60.0       63
80.0       12
67.0        6
Name: district_code, dtype: int64

In [10]:
# Arrange data into X features matrix and y target vector 
X_train = train[features]
y_train = train[target]
X_test = test[features]


In [11]:
X_train.head(20)

,amount_tsh,date_recorded,gps_height,longitude,latitude,region_code,district_code,population,construction_year,years_since_constr,basin,region,public_meeting,recorded_by,scheme_management,permit,extraction_type_class,management,payment,water_quality,quantity_group,source,source_class,waterpoint_type,water_quality_extraction_method,quality_and_quantity,Year_built_insufficientsupply,Year_built_sufficientsupply,extraction_gravity_insufficent
0,6000.0,2011,1390.0,34.938093,-9.856322,11,5.0,109.0,1999.0,12.0,Lake Nyasa,Iringa,True,GeoData Consultants Ltd,VWC,False,gravity,vwc,pay annually,soft,enough,spring,groundwater,communal standpipe,False,False,True,True,False
1,NaN,2013,1399.0,34.698766,-2.147466,20,2.0,280.0,2010.0,3.0,Lake Victoria,Mara,NaN,GeoData Consultants Ltd,Other,True,gravity,wug,never pay,soft,insufficient,rainwater harvesting,surface,communal standpipe,False,False,True,True,True
2,25.0,2013,686.0,37.460664,-3.821329,21,4.0,250.0,2009.0,4.0,Pangani,Manyara,True,GeoData Consultants Ltd,VWC,True,gravity,vwc,pay per bucket,soft,enough,dam,surface,communal standpipe multiple,False,False,False,True,False
3,NaN,2013,263.0,38.486161,-11.155298,90,63.0,58.0,1986.0,27.0,Ruvuma / Southern Coast,Mtwara,True,GeoData Consultants Ltd,VWC,True,submersible,vwc,never pay,soft,dry,machine dbh,groundwater,communal standpipe multiple,False,True,True,False,False
4,NaN,2011,NaN,31.130847,-1.825359,18,1.0,NaN,NaN,NaN,Lake Victoria,Kagera,True,GeoData Consultants Ltd,NaN,True,gravity,other,never pay,soft,seasonal,rainwater harvesting,surface,communal standpipe,False,False,False,False,False
5,20.0,2011,NaN,39.172796,-4.765587,4,8.0,1.0,2009.0,2.0,Pangani,Tanga,True,GeoData Consultants Ltd,VWC,True,submersible,vwc,pay per bucket,salty,enough,other,unknown,communal standpipe multiple,False,True,False,True,False
6,NaN,2012,NaN,33.362410,-3.766365,17,3.0,NaN,NaN,NaN,Internal,Shinyanga,True,GeoData Consultants Ltd,VWC,True,handpump,vwc,never pay,soft,enough,machine dbh,groundwater,hand pump,False,False,False,True,False
7,NaN,2012,NaN,32.620617,-4.226198,17,3.0,NaN,NaN,NaN,Lake Tanganyika,Shinyanga,True,GeoData Consultants Ltd,NaN,True,handpump,wug,unknown,milky,enough,shallow well,groundwater,hand pump,False,False,False,True,False
8,NaN,2012,NaN,32.711100,-5.146712,14,6.0,NaN,NaN,NaN,Lake Tanganyika,Tabora,True,GeoData Consultants Ltd,VWC,True,handpump,vwc,never pay,salty,seasonal,machine dbh,groundwater,hand pump,False,True,False,False,False
9,NaN,2011,NaN,30.626991,-1.257051,18,1.0,NaN,NaN,NaN,Lake Victoria,Kagera,True,GeoData Consultants Ltd,NaN,True,handpump,vwc,never pay,soft,enough,shallow well,groundwater,hand pump,False,False,False,True,False


In [12]:
extractionyear = X_train[X_train['quantity_group']== 'enough']

In [13]:
extractionyear2 = X_train[X_train['quantity_group']== 'insufficient']

In [14]:
extractionyear.describe()

,amount_tsh,date_recorded,gps_height,longitude,latitude,region_code,district_code,population,construction_year,years_since_constr
count,12083.000000,33186.000000,22698.000000,32260.000000,32260.000000,33186.000000,33166.000000,21630.000000,22334.000000,22334.000000
mean,1138.249727,2011.869825,1068.313155,35.059941,-5.925161,15.639306,5.392872,273.389968,1998.170278,13.899704
std,5382.914870,0.951556,633.226701,2.668298,2.784229,17.643923,9.578339,582.568197,11.922081,11.910357
min,0.200000,2004.000000,-90.000000,29.607201,-11.648378,1.000000,1.000000,1.000000,1960.000000,-7.000000
25%,50.000000,2011.000000,461.000000,33.279879,-8.671304,6.000000,1.000000,36.000000,1990.000000,4.000000
50%,250.000000,2011.000000,1219.000000,34.908172,-5.449370,12.000000,3.000000,144.000000,2002.000000,11.000000
75%,1000.000000,2013.000000,1555.000000,37.199071,-3.357421,17.000000,4.000000,300.000000,2008.000000,22.000000
max,250000.000000,2013.000000,2628.000000,40.344301,-1.041963,99.000000,80.000000,11463.000000,2013.000000,53.000000


In [15]:
extractionyear2

,amount_tsh,date_recorded,gps_height,longitude,latitude,region_code,district_code,population,construction_year,years_since_constr,basin,region,public_meeting,recorded_by,scheme_management,permit,extraction_type_class,management,payment,water_quality,quantity_group,source,source_class,waterpoint_type,water_quality_extraction_method,quality_and_quantity,Year_built_insufficientsupply,Year_built_sufficientsupply,extraction_gravity_insufficent
1,NaN,2013,1399.0,34.698766,-2.147466,20,2.0,280.0,2010.0,3.0,Lake Victoria,Mara,NaN,GeoData Consultants Ltd,Other,True,gravity,wug,never pay,soft,insufficient,rainwater harvesting,surface,communal standpipe,False,False,True,True,True
11,200.0,2013,1062.0,35.770258,-10.574175,10,5.0,250.0,1987.0,26.0,Lake Nyasa,Ruvuma,True,GeoData Consultants Ltd,WUG,False,handpump,wug,pay when scheme fails,soft,insufficient,shallow well,groundwater,hand pump,False,False,True,False,False
18,NaN,2011,1162.0,32.920154,-1.947868,19,1.0,1000.0,1999.0,12.0,Lake Victoria,Mwanza,NaN,GeoData Consultants Ltd,NaN,True,other,vwc,never pay,milky,insufficient,spring,groundwater,other,False,False,True,False,False
28,NaN,2013,1443.0,37.611126,-3.263526,3,1.0,1.0,2000.0,13.0,Pangani,Kilimanjaro,True,GeoData Consultants Ltd,Company,True,gravity,company,pay monthly,soft,insufficient,spring,groundwater,communal standpipe,False,False,True,False,True
30,NaN,2011,NaN,36.108312,-5.791000,1,4.0,NaN,NaN,NaN,Wami / Ruvu,Dodoma,True,GeoData Consultants Ltd,VWC,True,handpump,vwc,never pay,soft,insufficient,shallow well,groundwater,hand pump,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59384,NaN,2011,NaN,30.667805,-2.483710,18,30.0,NaN,NaN,NaN,Lake Victoria,Kagera,True,GeoData Consultants Ltd,VWC,False,gravity,vwc,never pay,soft,insufficient,spring,groundwater,communal standpipe,False,False,True,False,True
59388,NaN,2013,1414.0,30.692400,-3.593827,16,1.0,500.0,1995.0,18.0,Lake Tanganyika,Kigoma,True,GeoData Consultants Ltd,VWC,True,gravity,vwc,unknown,soft,insufficient,spring,groundwater,improved spring,False,False,True,False,True
59390,NaN,2011,1715.0,31.370848,-8.258160,15,2.0,150.0,1991.0,20.0,Lake Tanganyika,Rukwa,True,GeoData Consultants Ltd,VWC,False,handpump,vwc,never pay,soft,insufficient,machine dbh,groundwater,hand pump,False,False,True,False,False
59393,NaN,2012,NaN,33.866852,-4.287410,14,2.0,NaN,NaN,NaN,Internal,Tabora,False,GeoData Consultants Ltd,Water authority,False,gravity,private operator,pay per bucket,soft,insufficient,dam,surface,other,False,False,True,False,True


In [16]:
y_train

0            functional
1            functional
2            functional
3        non functional
4            functional
              ...      
59395        functional
59396        functional
59397        functional
59398        functional
59399        functional
Name: status_group, Length: 59400, dtype: object

In [17]:
X_train.describe()

,amount_tsh,date_recorded,gps_height,longitude,latitude,region_code,district_code,population,construction_year,years_since_constr
count,17761.000000,59400.000000,38962.000000,57588.000000,57588.000000,59400.000000,59377.000000,38019.000000,38691.000000,38691.000000
mean,1062.351942,2011.921667,1018.860839,35.149669,-5.885572,15.297003,5.631928,281.087167,1996.814686,15.355742
std,5409.344940,0.958758,612.566092,2.607428,2.809876,17.587406,9.634877,564.687660,12.472045,12.492673
min,0.200000,2002.000000,-90.000000,29.607122,-11.649440,1.000000,1.000000,1.000000,1960.000000,-7.000000
25%,50.000000,2011.000000,393.000000,33.285100,-8.643841,5.000000,2.000000,40.000000,1987.000000,5.000000
50%,250.000000,2012.000000,1167.000000,35.005943,-5.172704,12.000000,3.000000,150.000000,2000.000000,13.000000
75%,1000.000000,2013.000000,1498.000000,37.233712,-3.372824,17.000000,5.000000,324.000000,2008.000000,25.000000
max,350000.000000,2013.000000,2770.000000,40.345193,-0.998464,99.000000,80.000000,30500.000000,2013.000000,53.000000


In [18]:
model2 = Pipeline([
                  ('ohe', OrdinalEncoder(drop_invariant=True)),
                  ('impute', SimpleImputer(strategy='median')),
                  ('scaler', StandardScaler()),
                  ('classifier', RandomForestClassifier(
                      min_samples_split = 4,
                      max_features = 9,
                      criterion = 'entropy',
                      n_estimators=100,
                      n_jobs=-1,
                      random_state=42,
                      max_depth=20)
                   )])
model2.fit(X_train, y_train)

print('training accuracy:', model2.score(X_train, y_train))


training accuracy: 0.9284006734006734


In [19]:
y_pred = model2.predict(X_test)

In [20]:
# from sklearn.metrics import plot_confusion_matrix
# from sklearn.metrics import classification_report
# plt.rcParams['figure.dpi'] =100
# plot_confusion_matrix(model2,X_train,y_train, values_format= '.0f', xticks_rotation= 'vertical')

correct predictions

In [21]:
cor_pred= 31752+2839+20466
cor_pred

55057

total predictions

In [22]:
tot_pred= cor_pred+1253+225+2286+72+375+132
tot_pred

59400

In [23]:
cor_pred/ tot_pred

0.9268855218855219

In [24]:
# print(classification_report(y_test,y_pred))

In [25]:
# DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
# sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')
# submission = sample_submission.copy()
# submission['status_group'] = y_pred
# submission.to_csv('your-submission-filename3.csv', index=False)

In [26]:
# from google.colab import files
# files.download('your-submission-filename3.csv')

In [27]:

# import matplotlib.pyplot as plt
# model = model2.named_steps['classifier']
# encoder = model2.named_steps['ohe']
# encoded_columns = encoder.transform(X_train).columns
# importances = pd.Series(model.feature_importances_, encoded_columns)
# plt.figure(figsize=(10,30))
# importances.sort_values().plot.barh()

In [28]:
# import matplotlib.pyplot as plt


In [29]:
# plt.figure(figsize=(10,10))
# importances.sort_values().plot.barh()

In [35]:
model = Pipeline([
                  ('ohe', OrdinalEncoder(drop_invariant=True)),
                  ('impute', SimpleImputer(strategy='median')),
                  ('scaler', StandardScaler()),
                  ('classifier', RandomForestClassifier(
                      n_estimators=100,
                      n_jobs=-1))
                  
])

param_distributions = {
    'classifier__min_samples_split' : (2,4,6,8),
    'classifier__criterion' : ('gini','entropy'),
    'classifier__max_features' : (2,4,6,8,9,10),
    'classifier__max_depth' : (10,20,30)
    }

search = RandomizedSearchCV(
    model,
    param_distributions=param_distributions,
    n_iter=100,
    cv=5,
    scoring='accuracy',
    verbose=1,
    return_train_score= True,
    n_jobs=-1
)

search.fit(X_train, y_train)

print('Cross Validation Best Score', search.best_score_)
print('Best Estimator:', search.best_params_)
print('Best Model', search.best_estimator_)


Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  7.5min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed: 32.7min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed: 75.3min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed: 86.5min finished


Cross Validation Best Score 0.8093434343434343
Best Estimator: {'classifier__min_samples_split': 6, 'classifier__max_features': 8, 'classifier__max_depth': 20, 'classifier__criterion': 'gini'}
Best Model Pipeline(memory=None,
         steps=[('ohe',
                 OrdinalEncoder(cols=['basin', 'region', 'public_meeting',
                                      'recorded_by', 'scheme_management',
                                      'permit', 'extraction_type_class',
                                      'management', 'payment', 'water_quality',
                                      'quantity_group', 'source',
                                      'source_class', 'waterpoint_type'],
                                drop_invariant=True, handle_missing='value',
                                handle_unknown='value',
                                mapping=[{'col': 'basin',
                                          'dat...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0